# Strojové učenie

# Zdroje
[Encoding] - https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931

[Stromy] - http://scikit-learn.org/stable/modules/tree.html

# Knižnice

In [108]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

import pydotplus
import collections

import category_encoders as ce
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.cross_validation import cross_val_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced
from collections import Counter


In [28]:
from sklearn.preprocessing import LabelEncoder

In [29]:
#Normalizujeme si kategoricke atributy na cisla
def encodeByNum(data):
    metadata = data.drop(['CLASS'], axis=1)
    columns = metadata.select_dtypes('object').columns
    metadata = pd.get_dummies(metadata, prefix=columns)
    metadata['CLASS'] = data['CLASS']
    return metadata

In [30]:
def init_dataset2(file, sep):
    # Načítame si súbor s datasetom so separátom ',' a ako index nastavime prvý stĺpec
    data = pd.read_csv(file, sep=sep, index_col=0)
    
    metadata = data.drop(['DATE_OF_BIRTH', 'CLASS'], axis=1)
    col_names = metadata.select_dtypes('object').columns
    metadata['CLASS'] = data['CLASS']
    
    #print(metadata.info())
    return metadata

In [31]:
def init_dataset(file):
    # Načítame si súbor s datasetom so separátom ',' a ako index nastavime prvý stĺpec
    data = pd.read_csv(file, sep=',', index_col=0)
    
    metadata = data.drop(['DATE_OF_BIRTH', 'CLASS'], axis=1)
    col_names = metadata.select_dtypes('object').columns
    metadata['CLASS'] = data['CLASS']
    
    #print(metadata.info())
    return metadata

In [32]:
def getDataSets(train_file, test_file):

    train_data = init_dataset(train_file)
    test_data = init_dataset(test_file)

    #Ziskame si pocet trenovacich dat, aby sme vedeli podla, ktoreho indexu ich neskôr rozdelime
    train_rows_count = train_data.shape[0]

    data = pd.concat([train_data, test_data])
    data.reset_index(drop=True)

    #Transformacia kategorickych na numericke
    data = encodeByNum(data)
    
    #Rozdelenie dat naspat na vstupne datsety
    train_data = data.iloc[:train_rows_count,:]
    test_data = data.iloc[train_rows_count+1:,:]
    
    return train_data, test_data

In [77]:
def getFeaturesLabels(data):
    #Rozdelenie dat na atribúty, ktoré budú trénované na predikovanie triedy
    X = data.drop('CLASS', axis=1)
    y = data['CLASS']
    return X, y

In [34]:
#Zistime si dolezitost atributov
def get_important_features_names(data, clf):
    important_features = []
    for name, importance in zip(data.columns, clf.feature_importances_):
        if importance > 0:
            important_features.append(name)
    return important_features

In [112]:
def prediction(train_X, train_y, test_X, test_y, params):
    pipeline = make_pipeline(SMOTE(random_state=0), DecisionTreeClassifier(**params))
    
    
    x_smote, y_smote = SMOTE().fit_sample(train_X, train_y)
    print("SMOTE data distribution: {}".format(Counter(y_smote)))
    
    pipeline.fit(x_smote, y_smote)
    
    print(classification_report_imbalanced(test_y, pipeline.predict(test_X)))
    
    
    
    '''
    clf = DecisionTreeClassifier(**params)
    #Natrénujeme model
    clf = clf.fit(train_X, train_y)
    scores = cross_val_score(clf, test_X, test_y, cv=10, scoring="f1_macro")
    
    #Predikujeme triedu pre testovacie dáta
    y_pred_test = clf.predict(test_X)
    y_pred_train = clf.predict(train_X)

    print(confusion_matrix(test_y, y_pred_test))
    print(classification_report(test_y, y_pred_test))
    
    print("Test data Precision: ", precision_score(test_y, y_pred_test, average='macro'))
    print("Train data Precision: ", precision_score(train_y, y_pred_train, average='macro'))

    print("Test data Recall: ", recall_score(test_y, y_pred_test, average='macro'))
    print("Train data Recall: ", recall_score(train_y, y_pred_train, average='macro'))

    print("Test data Accuracy: ", accuracy_score(test_y, y_pred_test))
    print("Train data Accuracy: ", accuracy_score(train_y, y_pred_train))

    print("\nJednotlivé f1_score pre každú iteráciu cross validácie - priemer:")
    print(scores.mean())
    '''
    return pipeline
    

In [61]:
def prediction_random_forest(train_X, train_y, test_X, test_y, params):
    clf = RandomForestClassifier(**params)
    #Natrénujeme model
    clf = clf.fit(train_X, train_y)
    scores = cross_val_score(clf, test_X, test_y, cv=10, scoring="f1_macro")
    
    #Predikujeme triedu pre testovacie dáta
    y_pred_test = clf.predict(test_X)
    y_pred_train = clf.predict(train_X)

    print(confusion_matrix(test_y, y_pred_test))
    print(classification_report(test_y, y_pred_test))
    
    print("Test data Precision: ", precision_score(test_y, y_pred_test, average='macro'))
    print("Train data Precision: ", precision_score(train_y, y_pred_train, average='macro'))

    print("Test data Recall: ", recall_score(test_y, y_pred_test, average='macro'))
    print("Train data Recall: ", recall_score(train_y, y_pred_train, average='macro'))

    print("Test data Accuracy: ", accuracy_score(test_y, y_pred_test))
    print("Train data Accuracy: ", accuracy_score(train_y, y_pred_train))

    
    print("\nJednotlivé f1_score pre každú iteráciu cross validácie - priemer:")
    print(scores.mean())
    return clf

# Doplnkové čistenie
- V tejto časti opisujeme doplnkove cistenie dat na zaklade revizie predchadzajuceho odovzdania

In [37]:
classifiers = pd.DataFrame() # tu sa budu ukladat vysledky jednotlivých klasifikatorov

## Výstupné súbory z fáze predspracovania

In [38]:
file_median_train = 'data/clean_train_data/ctrain_median.csv'
file_modus_train = 'data/clean_train_data/ctrain_modus.csv'
file_mean_train = 'data/clean_train_data/ctrain_mean.csv'
file_knn_train = 'data/clean_train_data/ctrain_knn.csv'

file_median_test = 'data/clean_test_data/ctest_median.csv'
file_modus_test = 'data/clean_test_data/ctest_modus.csv'
file_mean_test = 'data/clean_test_data/ctest_mean.csv'
file_knn_test = 'data/clean_test_data/ctest_knn.csv'

## Inicializácia DataFramov

In [86]:
data_median_train, data_median_test = getDataSets(file_median_train, file_median_test)
data_modus_train, data_modus_test = getDataSets(file_modus_train, file_modus_test)
data_mean_train, data_mean_test = getDataSets(file_mean_train, file_mean_test)
data_knn_train, data_knn_test = getDataSets(file_knn_train, file_knn_test)

## Priprava dát na klasifikáciu

In [87]:
train_median_X, train_median_y = getFeaturesLabels(data_median_train)
test_median_X, test_median_y = getFeaturesLabels(data_median_test)

train_modus_X, train_modus_y = getFeaturesLabels(data_modus_train)
test_modus_X, test_modus_y = getFeaturesLabels(data_modus_test)

train_mean_X, train_mean_y = getFeaturesLabels(data_mean_train)
test_mean_X, test_mean_y = getFeaturesLabels(data_mean_test)

train_knn_X, train_knn_y = getFeaturesLabels(data_knn_train)
test_knn_X, test_knn_y = getFeaturesLabels(data_knn_test)

In [41]:
#Inicializacia datasetu so zjednotenými nazvami
test_file = "data/zadanie4/ctest_revision.csv"
train_file = "data/zadanie4/ctrain.csv"

train_data = init_dataset2(train_file, '\t')
test_data = init_dataset2(test_file, '\t')

#Ziskame si pocet trenovacich dat, aby sme vedeli podla, ktoreho indexu ich neskôr rozdelime
train_rows_count = train_data.shape[0]

data = pd.concat([train_data, test_data])
data.reset_index(drop=True)

#Transformacia kategorickych na numericke
data = encodeByNum(data)

#Rozdelenie dat naspat na vstupne datsety
train_data = data.loc[:train_rows_count+1,:]
test_data = data.loc[train_rows_count:,:]

In [42]:
#Rozdelenie dat na atribúty, ktoré budú trénované na predikovanie triedy
train_X = train_data.drop('CLASS', axis=1)
train_y = train_data['CLASS']
#Rozdelenie testovacich dat na predikciu a porovnanie vysledkov predikcie
test_X = test_data.drop('CLASS', axis=1)
test_y = test_data['CLASS']

# Zistujeme najlepsie parametre a atributy

In [85]:
#Vytvoríme Decision tree
clf = DecisionTreeClassifier(random_state=0)
cross_validation = 10 # násobná crossvalidácia

#Nastavujeme hyperparametre
cv_params = {'class_weight': ['balanced'], 'max_depth': [1,2,3,4] + list(range(5,10,2)), 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1,2,3,4] + list(range(5,10,2))}
optimization = GridSearchCV(clf,
                            cv_params, 
                             scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 

#Inicializacia grid searchu
optimization_median = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 
optimization_modus = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 
optimization_mean = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 
optimization_knn = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 

<strong>max_depth</strong> - maximálna hĺbka stromu<br>
<strong>criterion</strong> - http://haohanw.blogspot.sk/2014/08/ml-decision-tree-rule-selection.html, The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain.<br>
<strong>min_samples_split</strong> - minimálne množstvo vzoriek na rozdelenie prvku v strome<br>
<strong>min_samples_leaf</strong> - minimálne množstvo vzoriek potrebných na jeden list<br>
<strong>class_weight</strong> - nastavenie váhy jednotlivých vied

Pri vyskúšaní hyperparametru splitter na random sme zistili, že to nie je správna cesta, pretože nám to veľmi znížilo skóre. Ak sme použili menšie množstvo atribútov (napr. max_features=4), ako 5 (zistlili sme si 5 najlepších parametrov) tak sa na nám skóre výrazne znížili.<br>
Pri znížení násobnosti cross validácie sa nám f1_score znížilo a minimálne sa nám takisto znížili aj pri použití default random state. <br>
Pri použití hyperparametru class_weight a nastavení ho na hodnotu balanced sme zaznamenali pokles skóre.

In [89]:
%%time
optimization.fit(train_X, train_y)

Fitting 10 folds for each of 98 candidates, totalling 980 fits


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

CPU times: user 2.11 s, sys: 248 ms, total: 2.36 s
Wall time: 21.1 s


[Parallel(n_jobs=-1)]: Done 980 out of 980 | elapsed:   20.9s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'class_weight': ['balanced'], 'max_depth': [1, 2, 3, 4, 5, 7, 9], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 3, 4, 5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=True)

In [90]:
%%time
optimization_median.fit(train_median_X, train_median_y)
optimization_modus.fit(train_modus_X, train_modus_y)
optimization_mean.fit(train_mean_X, train_mean_y)
optimization_knn.fit(train_knn_X, train_knn_y)

Fitting 10 folds for each of 98 candidates, totalling 980 fits


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

...

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

CPU times: user 8.66 s, sys: 1.72 s, total: 10.4 s
Wall time: 1min 21s


[Parallel(n_jobs=-1)]: Done 980 out of 980 | elapsed:   19.9s finished


In [67]:
list(filter(lambda x: 'best' in x, dir(optimization)))

['best_estimator_', 'best_index_', 'best_params_', 'best_score_']

In [68]:
#Vypiseme si najlepsie zistene parametre a score
print('median')
print(optimization_median.best_params_)
print(optimization_median.best_score_)
print('modus')
print(optimization_modus.best_params_)
print(optimization_modus.best_score_)
print('mean')
print(optimization_mean.best_params_)
print(optimization_mean.best_score_)
print('knn')
print(optimization_knn.best_params_)
print(optimization_knn.best_score_)

median
{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 3}
0.8786650950880928
modus
{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}
0.9037768259605322
mean
{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}
0.8870317159658619
knn
{'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 3}
0.8954079215343964


In [69]:
#Vypiseme si najlepsie zistene parametre a score
print('median')
print(optimization_median.best_params_)
print(optimization_median.best_score_)
print('modus')
print(optimization_modus.best_params_)
print(optimization_modus.best_score_)
print('mean')
print(optimization_mean.best_params_)
print(optimization_mean.best_score_)
print('knn')
print(optimization_knn.best_params_)
print(optimization_knn.best_score_)

median
{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 3}
0.8786650950880928
modus
{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}
0.9037768259605322
mean
{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 2}
0.8870317159658619
knn
{'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 3}
0.8954079215343964


In [70]:
# Zistime si najlepsie parametre pre DecisionTree
best_params = optimization.best_params_

In [71]:
# Zistime najlepsie skore pre nase testovacie data
best_score = optimization.best_score_

In [72]:
best_features_median = get_important_features_names(train_median_X, optimization_median.best_estimator_)
best_features_modus = get_important_features_names(train_modus_X, optimization_modus.best_estimator_)
best_features_mean = get_important_features_names(train_mean_X, optimization_mean.best_estimator_)
best_features_knn = get_important_features_names(train_knn_X, optimization_knn.best_estimator_)

### Model s doplnenými chybajúcimi hodnotami podľa Modus 

In [113]:
# Zistime si najlepsie atributy
best_features = best_features_modus
best_params = optimization_modus.best_params_

print("Best_features: ", best_features)
print("best_params: ", best_params)
#Natrenovanie modelu podla zistených
train_best_data = train_modus_X.loc[:, best_features]
test_best_data = test_modus_X.loc[:, best_features]

print("\nPredikcia s parametrami - najlepšie atribúty")
result = prediction(train_best_data, train_modus_y, test_best_data, test_modus_y, best_params)
print("\nPredikcia bez parametrov - všetky atribúty")
result = prediction(train_modus_X, train_modus_y, test_modus_X, test_modus_y, {})

Best_features:  ['ON_THYROXINE', 'TSH', 'T3', 'FTI', 'HOURS_PER_WEEK']
best_params:  {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1}

Predikcia s parametrami - najlepšie atribúty
SMOTE data distribution: Counter({'REPLACEMENT_THERAPY': 2713, 'NEGATIVE': 2713, 'UNDERREPLACEMENT': 2713, 'OVERREPLACEMENT': 2713})
                           pre       rec       spe        f1       geo       iba       sup

           NEGATIVE       1.00      0.97      0.97      0.99      0.97      0.95       934
    OVERREPLACEMENT       0.64      0.82      0.99      0.72      0.90      0.80        11
REPLACEMENT_THERAPY       0.14      0.44      0.97      0.21      0.66      0.41         9
   UNDERREPLACEMENT       0.89      1.00      1.00      0.94      1.00      1.00        17

        avg / total       0.99      0.97      0.97      0.97      0.97      0.94       971


Predikcia bez parametrov - všetky atribúty
SMOTE data distribution: Counter({'REPLACEMENT_THER

Stratégia dopĺňania chýbajúcich hodnôt pomocou modusu sa zdá byť najvhodnejšia, pretože vykazuje najvyššiu hodnotu **správnosti (accuracy)**. V našom prípade nás ale accuracy nezaujíma (pretože máme nevyvážené triedy - dominancia  NEGATIVE), ale zaujíma nás najmä f1_score.

### Model s doplnenými chybajúcimi hodnotami podľa KNN 

In [63]:
# Zistime si najlepsie atributy
best_features = best_features_knn
best_params = optimization_knn.best_params_

print("Best_features: ", best_features)
print("best_params: ", best_params)

#Natrenovanie modelu podla zistených
train_best_data = train_knn_X.loc[:, best_features]
test_best_data = test_knn_X.loc[:, best_features]

print("\nPredikcia s parametrami - najlepšie atribúty")
result = prediction(train_best_data, train_knn_y, test_best_data, test_knn_y, best_params)
print("\nPredikcia bez parametrov - všetky atribúty")
result = prediction(train_knn_X, train_knn_y, test_knn_X, test_knn_y, {})

Best_features:  ['AGE', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'HOURS_PER_WEEK', 'ON_THYROXINE', 'FTI_MEASURED', 'REFERRAL_SOURCE_SVI']
best_params:  {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1}

Predikcia s parametrami - najlepšie atribúty
[[930   2   2   0]
 [  1   9   1   0]
 [  2   4   3   0]
 [  0   0   0  17]]
                     precision    recall  f1-score   support

           NEGATIVE       1.00      1.00      1.00       934
    OVERREPLACEMENT       0.60      0.82      0.69        11
REPLACEMENT_THERAPY       0.50      0.33      0.40         9
   UNDERREPLACEMENT       1.00      1.00      1.00        17

        avg / total       0.99      0.99      0.99       971

Test data Precision:  0.7741961414790997
Train data Precision:  0.9251769626769627
Test data Recall:  0.7868081240672248
Train data Recall:  0.9906421190436849
Test data Accuracy:  0.9876416065911432
Train data Accuracy:  0.9967857142857143

Jednotlivé f1_score pre každú

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/s

[[932   1   1   0]
 [  0   9   2   0]
 [  1   4   4   0]
 [  0   0   0  17]]
                     precision    recall  f1-score   support

           NEGATIVE       1.00      1.00      1.00       934
    OVERREPLACEMENT       0.64      0.82      0.72        11
REPLACEMENT_THERAPY       0.57      0.44      0.50         9
   UNDERREPLACEMENT       1.00      1.00      1.00        17

        avg / total       0.99      0.99      0.99       971

Test data Precision:  0.8033034757311285
Train data Precision:  1.0
Test data Recall:  0.815121233750784
Train data Recall:  1.0
Test data Accuracy:  0.9907312049433573
Train data Accuracy:  1.0

Jednotlivé f1_score pre každú iteráciu cross validácie - priemer:
0.6406012556322154


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Model s doplnenými chybajúcimi hodnotami podľa Mean 

In [ ]:
# Zistime si najlepsie atributy
best_features = best_features_mean
best_params = optimization_mean.best_params_

print("Best_features: ", best_features)
print("best_params: ", best_params)

#Natrenovanie modelu podla zistených
train_best_data = train_mean_X.loc[:, best_features]
test_best_data = test_mean_X.loc[:, best_features]

print("\nPredikcia s parametrami - najlepšie atribúty")
result = prediction(train_best_data, train_mean_y, test_best_data, test_mean_y, best_params)
print("\nPredikcia bez parametrov - všetky atribúty")
result = prediction(train_mean_X, train_mean_y, test_mean_X, test_mean_y, {})

### Model s doplnenými chybajúcimi hodnotami podľa Median 

In [ ]:
# Zistime si najlepsie atributy
best_features = best_features_median
best_params = optimization_median.best_params_

print("Best_features: ", best_features)
print("best_params: ", best_params)

#Natrenovanie modelu podla zistených
train_best_data = train_median_X.loc[:, best_features]
test_best_data = test_median_X.loc[:, best_features]

print("\nPredikcia s parametrami - najlepšie atribúty")
result = prediction(train_best_data, train_median_y, test_best_data, test_median_y, best_params)
print("\nPredikcia bez parametrov - všetky atribúty")
result = prediction(train_median_X, train_median_y, test_median_X, test_median_y, {})

Doplnenie medianom sme zistli že nie je najvhodnejšie riešenie, kvôli nízkemu **recall**. Takisto vykazuje aj nanižšiu **správnosť (accuracy)**, čiže túto metódu by sme nepoužili. 

In [ ]:
#Vypiseme si najlepsie atributy pre jednotlive predikovanie podla datasetov
print(best_features)

In [ ]:
#Natrenovanie modelu podla zistených
train_best_data = train_X
test_best_data = test_X
# TODO
result = prediction(train_best_data, train_y, test_best_data, test_y, {})

In [ ]:
test_best_data.info()

In [ ]:
print("Predikovanie na trenovacich datach: ",result.score(train_best_data, train_y))
print("Predikovanie na testovacich datach: ",result.score(test_best_data, test_y))
classifiers = classifiers.append([["Decision tree", result.score(train_best_data, train_y), result.score(test_best_data, test_y)]])

In [ ]:
# Visualize data
from sklearn.tree import export_graphviz
dot_data = export_graphviz(result, out_file=None, 
                         feature_names=best_features,  
                         class_names=train_y.unique(),  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render("img/tree_best") 
graph

# KNN klasifikátor

In [ ]:
cls = KNeighborsClassifier(6)
cls.fit(train_X, train_y)
y_pred_train = cls.predict(train_X)
y_pred_test = cls.predict(test_X)
print(cls)

In [ ]:
print("Predikovanie na trenovacich datach: ",accuracy_score(train_y, y_pred_train))
print("Predikovanie na testovacich datach: ",accuracy_score(test_y, y_pred_test))
classifiers = classifiers.append([["KNeighbors", accuracy_score(train_y, y_pred_train), accuracy_score(test_y, y_pred_test)]])

In [ ]:
print(confusion_matrix(test_y, y_pred_test))
print(classification_report(test_y, y_pred_test))

# Random Forest Classifier

Zvolili sme nepárny počet stromov, aby jeden bol vždy majoritný.

In [74]:
#Vytvoríme Decision tree
clf = RandomForestClassifier(random_state=0)
cross_validation = 10 # násobná crossvalidácia

#Nastavujeme hyperparametre
cv_params = {'class_weight': ['balanced'], 'n_estimators': list(range(11, 120, 10))}
optimization = GridSearchCV(clf,
                            cv_params, 
                             scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 

#Inicializacia grid searchu
optimization_median = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 
optimization_modus = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 
optimization_mean = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 
optimization_knn = GridSearchCV(clf, cv_params, scoring = 'f1_macro', cv = cross_validation, n_jobs = -1, verbose=True) 

In [75]:
%%time
optimization_median.fit(train_median_X, train_median_y)
optimization_modus.fit(train_modus_X, train_modus_y)
optimization_mean.fit(train_mean_X, train_mean_y)
optimization_knn.fit(train_knn_X, train_knn_y)

Fitting 10 folds for each of 11 candidates, totalling 110 fits


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   13.6s finished


...

[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   12.4s finished
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:113

CPU times: user 2.31 s, sys: 628 ms, total: 2.93 s
Wall time: 53.7 s


[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   14.4s finished


In [76]:
# Zistime si najlepsie atributy
best_features = best_features_modus
best_params = optimization_modus.best_params_

print("Best_features: ", best_features)
print("best_params: ", best_params)
#Natrenovanie modelu podla zistených
train_best_data = train_modus_X.loc[:, best_features]
test_best_data = test_modus_X.loc[:, best_features]

print("\nPredikcia s parametrami - najlepšie atribúty")
result = prediction_random_forest(train_best_data, train_modus_y, test_best_data, test_modus_y, best_params)
print("\nPredikcia bez parametrov - všetky atribúty")
result = prediction_random_forest(train_modus_X, train_modus_y, test_modus_X, test_modus_y, {})

Best_features:  ['ON_THYROXINE', 'TSH', 'T3', 'FTI', 'HOURS_PER_WEEK']
best_params:  {'class_weight': 'balanced', 'n_estimators': 41}

Predikcia s parametrami - najlepšie atribúty


/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/s

[[932   0   2   0]
 [  0  10   1   0]
 [  1   5   3   0]
 [  0   0   0  17]]
                     precision    recall  f1-score   support

           NEGATIVE       1.00      1.00      1.00       934
    OVERREPLACEMENT       0.67      0.91      0.77        11
REPLACEMENT_THERAPY       0.50      0.33      0.40         9
   UNDERREPLACEMENT       1.00      1.00      1.00        17

        avg / total       0.99      0.99      0.99       971

Test data Precision:  0.7913987138263665
Train data Precision:  1.0
Test data Recall:  0.8100707287002791
Train data Recall:  1.0
Test data Accuracy:  0.9907312049433573
Train data Accuracy:  1.0

Jednotlivé f1_score pre každú iteráciu cross validácie - priemer:
0.673936064123634

Predikcia bez parametrov - všetky atribúty


/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/s

[[934   0   0   0]
 [  5   6   0   0]
 [  7   1   1   0]
 [ 14   0   0   3]]
                     precision    recall  f1-score   support

           NEGATIVE       0.97      1.00      0.99       934
    OVERREPLACEMENT       0.86      0.55      0.67        11
REPLACEMENT_THERAPY       1.00      0.11      0.20         9
   UNDERREPLACEMENT       1.00      0.18      0.30        17

        avg / total       0.97      0.97      0.96       971

Test data Precision:  0.9575148809523809
Train data Precision:  0.9996319470003681
Test data Recall:  0.4582590612002377
Train data Recall:  0.966224030841722
Test data Accuracy:  0.972193614830072
Train data Accuracy:  0.9985714285714286

Jednotlivé f1_score pre každú iteráciu cross validácie - priemer:
0.44632111154245574


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
print("Accuracy na train zložke: {:f}".format(cls.score(train_X, train_y)))
print("Accuracy na test zložke: {:f}".format(cls.score(test_X, test_y)))


In [ ]:
test_preds = cls.predict(test_X)

In [ ]:
classifiers = classifiers.append([["Random Forest", accuracy_score(train_y, train_preds), cls.score(test_X, test_y)]])

In [ ]:
print(classification_report(test_y, test_preds))

In [ ]:
for est in cls.estimators_:
    dot_data = export_graphviz(est, out_file=None,  
                             class_names=train_y.unique(),  
                             filled=True, rounded=True,  
                             special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph.render("img/random_forest") 

In [ ]:
graph

# Porovnanie klasifikátorov

V tejto časti sa budeme venovať porovnaniu výsledkov jednotlivých klasifikátorov. 

In [ ]:
classifiers.columns = ['CLASSIFIER_NAME', 'TRAIN_ACCURACY', 'TEST_ACCURACY']
classifiers